# Phase 2: Visual Reference Detection (English)
## Detects when speaker references visual elements

This notebook:
- Loads Phase 1 transcript (from Whisper)
- Detects visual references using regex + BERT
- Outputs timestamped reference points

**Uses:** Regex patterns + sentence-transformers for semantic matching

In [ ]:
# ====================================================================
# CELL 1: Install Dependencies
# ====================================================================
print("📦 Installing dependencies...")
!pip install -q sentence-transformers torch numpy
print("✅ Dependencies installed!")

In [ ]:
# ====================================================================
# CELL 2: Import Libraries
# ====================================================================
import os
import json
import re
from typing import List, Dict, Any, Tuple
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
import gc

print(f"🖥️  Device: {'CUDA (GPU)' if torch.cuda.is_available() else 'CPU'}")
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
# ====================================================================
# CELL 3: Configuration
# ====================================================================

CONFIG = {
    # Input from Phase 1 (change this to your file)
    "phase1_file": "/kaggle/working/output/transcript_XXXXX.json",  # ⚠️ UPDATE THIS
    
    # Reference detection settings
    "use_bert": True,  # Use semantic matching (recommended)
    "bert_model": "all-MiniLM-L6-v2",  # Fast, lightweight model
    "similarity_threshold": 0.45,  # Lower = more sensitive (0.3-0.6)
    
    # English reference phrases to detect
    "reference_phrases": [
        # Direct visual references
        "look at this",
        "as you can see",
        "shown here",
        "in this diagram",
        "this figure",
        "this image",
        "this graph",
        "this chart",
        "this table",
        "this equation",
        "this formula",
        
        # Pointing/showing
        "here you can see",
        "if you look at",
        "take a look at",
        "let me show you",
        "I'll show you",
        "notice here",
        "observe that",
        
        # Diagram/visual elements
        "on the screen",
        "on the board",
        "in the picture",
        "in the video",
        "this example",
        "this visualization",
        
        # Positional
        "on the left",
        "on the right",
        "at the top",
        "at the bottom",
        "in the middle",
        "over here",
    ],
    
    "output_dir": "/kaggle/working/output"
}

os.makedirs(CONFIG['output_dir'], exist_ok=True)
print("✅ Configuration loaded")
print(f"   BERT: {'Enabled' if CONFIG['use_bert'] else 'Disabled'}")
print(f"   Similarity threshold: {CONFIG['similarity_threshold']}")
print(f"   Reference phrases: {len(CONFIG['reference_phrases'])}")

In [ ]:
# ====================================================================
# CELL 4: Load Phase 1 Output
# ====================================================================

print(f"📂 Loading Phase 1 transcript from: {CONFIG['phase1_file']}")

with open(CONFIG['phase1_file'], 'r', encoding='utf-8') as f:
    phase1_data = json.load(f)

utterances = phase1_data['utterances']
video_id = phase1_data['video_id']

print(f"✅ Loaded transcript:")
print(f"   Video ID: {video_id}")
print(f"   Utterances: {len(utterances)}")
print(f"   Duration: {phase1_data['duration_ms']/1000:.1f}s")
print(f"\n📝 First utterance: {utterances[0]['text'][:100]}...")

In [ ]:
# ====================================================================
# CELL 5: Reference Detector Class
# ====================================================================

class EnglishReferenceDetector:
    """Detect visual references in English transcripts."""
    
    def __init__(self, phrases: List[str], model_name: str = None,
                 use_bert: bool = True, threshold: float = 0.45):
        """
        Initialize detector.
        
        Args:
            phrases: List of reference phrases to detect
            model_name: Sentence transformer model
            use_bert: Whether to use semantic matching
            threshold: Similarity threshold (0.0-1.0)
        """
        self.phrases = phrases
        self.use_bert = use_bert
        self.threshold = threshold
        self.model = None
        
        # Compile regex patterns (case-insensitive)
        self.patterns = [
            re.compile(r'\b' + re.escape(phrase) + r'\b', re.IGNORECASE)
            for phrase in phrases
        ]
        
        # Load semantic model if requested
        if use_bert and model_name:
            print(f"🤖 Loading semantic model: {model_name}")
            device = "cuda" if torch.cuda.is_available() else "cpu"
            self.model = SentenceTransformer(model_name, device=device)
            print(f"✅ Model loaded on {device}")
    
    def detect_with_regex(self, utterances: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Detect references using regex pattern matching."""
        print("🔍 Detecting with regex patterns...")
        references = []
        reference_id = 1
        
        for utt_idx, utterance in enumerate(utterances):
            text = utterance['text']
            
            for phrase, pattern in zip(self.phrases, self.patterns):
                match = pattern.search(text)
                
                if match:
                    references.append({
                        "reference_id": f"REF_{reference_id:03d}",
                        "text": text,
                        "timestamp_ms": utterance['start_ms'],
                        "end_ms": utterance['end_ms'],
                        "matched_phrase": phrase,
                        "reference_type": self._classify_type(phrase),
                        "detection_method": "regex",
                        "confidence": 0.95,
                        "context_before": self._get_context(utterances, utt_idx, -1),
                        "context_after": self._get_context(utterances, utt_idx, 1)
                    })
                    reference_id += 1
                    break  # Only count once per utterance
        
        print(f"   Found {len(references)} regex matches")
        return references
    
    def detect_with_semantic(self, utterances: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Detect references using semantic similarity."""
        if not self.model:
            print("⚠️  Semantic model not loaded, skipping")
            return []
        
        print(f"🧠 Detecting with semantic matching (threshold={self.threshold})...")
        references = []
        reference_id = 1000  # Different ID range for semantic
        
        # Encode reference phrases
        print("   Encoding reference phrases...")
        reference_embeddings = self.model.encode(
            self.phrases,
            convert_to_tensor=True,
            show_progress_bar=False
        )
        
        # Process in batches to save memory
        batch_size = 32
        print(f"   Processing {len(utterances)} utterances in batches...")
        
        for batch_start in range(0, len(utterances), batch_size):
            batch_end = min(batch_start + batch_size, len(utterances))
            batch = utterances[batch_start:batch_end]
            
            # Encode batch
            texts = [u['text'] for u in batch]
            text_embeddings = self.model.encode(
                texts,
                convert_to_tensor=True,
                show_progress_bar=False
            )
            
            # Calculate similarities
            similarities = torch.mm(text_embeddings, reference_embeddings.T)
            max_sims, _ = similarities.max(dim=1)
            
            # Find matches
            for i, (utterance, sim) in enumerate(zip(batch, max_sims)):
                if sim.item() >= self.threshold:
                    utt_idx = batch_start + i
                    references.append({
                        "reference_id": f"SEM_{reference_id:03d}",
                        "text": utterance['text'],
                        "timestamp_ms": utterance['start_ms'],
                        "end_ms": utterance['end_ms'],
                        "matched_phrase": "semantic_match",
                        "reference_type": "contextual",
                        "detection_method": "semantic",
                        "confidence": float(sim.item()),
                        "context_before": self._get_context(utterances, utt_idx, -1),
                        "context_after": self._get_context(utterances, utt_idx, 1)
                    })
                    reference_id += 1
        
        # Clean up
        del reference_embeddings, text_embeddings
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
        print(f"   Found {len(references)} semantic matches")
        return references
    
    def _classify_type(self, phrase: str) -> str:
        """Classify reference type based on phrase."""
        phrase_lower = phrase.lower()
        
        if 'diagram' in phrase_lower or 'figure' in phrase_lower:
            return 'diagram'
        elif 'equation' in phrase_lower or 'formula' in phrase_lower:
            return 'equation'
        elif 'graph' in phrase_lower or 'chart' in phrase_lower:
            return 'graph'
        elif 'table' in phrase_lower:
            return 'table'
        elif 'screen' in phrase_lower or 'board' in phrase_lower:
            return 'screen'
        else:
            return 'visual'
    
    def _get_context(self, utterances: List[Dict], index: int, offset: int) -> str:
        """Get context utterance."""
        target_idx = index + offset
        if 0 <= target_idx < len(utterances):
            return utterances[target_idx]['text']
        return ""

print("✅ Detector class defined")

In [ ]:
# ====================================================================
# CELL 6: Run Detection
# ====================================================================

print("="*70)
print("🚀 STARTING REFERENCE DETECTION")
print("="*70)

# Initialize detector
detector = EnglishReferenceDetector(
    phrases=CONFIG['reference_phrases'],
    model_name=CONFIG['bert_model'] if CONFIG['use_bert'] else None,
    use_bert=CONFIG['use_bert'],
    threshold=CONFIG['similarity_threshold']
)

# Step 1: Regex detection (fast, high precision)
print("\n[1/3] Regex pattern matching...")
regex_refs = detector.detect_with_regex(utterances)

# Step 2: Semantic detection (slower, catches more)
semantic_refs = []
if CONFIG['use_bert']:
    print("\n[2/3] Semantic matching...")
    semantic_refs = detector.detect_with_semantic(utterances)

# Step 3: Merge and deduplicate
print("\n[3/3] Merging results...")
all_refs = regex_refs + semantic_refs

# Remove duplicates (within 2 second window)
unique_refs = []
seen_windows = set()

for ref in sorted(all_refs, key=lambda x: (x['timestamp_ms'], -x['confidence'])):
    # Use 2-second windows
    window = ref['timestamp_ms'] // 2000
    
    if window not in seen_windows:
        unique_refs.append(ref)
        seen_windows.add(window)

print(f"\n✅ Detection complete!")
print(f"   Regex matches: {len(regex_refs)}")
print(f"   Semantic matches: {len(semantic_refs)}")
print(f"   Unique references: {len(unique_refs)}")

In [ ]:
# ====================================================================
# CELL 7: Save Results
# ====================================================================

result = {
    "video_id": video_id,
    "video_url": phase1_data.get('video_url', ''),
    "references": unique_refs,
    "reference_count": len(unique_refs),
    "detection_summary": {
        "regex_count": len(regex_refs),
        "semantic_count": len(semantic_refs),
        "unique_count": len(unique_refs),
        "method": "regex+semantic" if CONFIG['use_bert'] else "regex_only"
    },
    "config": {
        "model": CONFIG['bert_model'] if CONFIG['use_bert'] else None,
        "threshold": CONFIG['similarity_threshold'],
        "phrase_count": len(CONFIG['reference_phrases'])
    }
}

# Save JSON
output_file = f"{CONFIG['output_dir']}/phase2_references_{video_id}.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

# Save simple text report
report_file = f"{CONFIG['output_dir']}/phase2_report_{video_id}.txt"
with open(report_file, 'w', encoding='utf-8') as f:
    f.write(f"Visual Reference Detection Report\n")
    f.write(f"Video ID: {video_id}\n")
    f.write(f"="*70 + "\n\n")
    
    for i, ref in enumerate(unique_refs, 1):
        time_sec = ref['timestamp_ms'] / 1000
        f.write(f"[{i}] {time_sec:.1f}s - {ref['detection_method']} (conf: {ref['confidence']:.2f})\n")
        f.write(f"    {ref['text']}\n\n")

file_size = os.path.getsize(output_file) / 1024

print("\n" + "="*70)
print("✅ PHASE 2 COMPLETE!")
print("="*70)
print(f"📊 Results:")
print(f"   - Total references: {len(unique_refs)}")
print(f"   - Average confidence: {np.mean([r['confidence'] for r in unique_refs]):.2f}")
print(f"\n💾 Files saved:")
print(f"   - JSON: {output_file} ({file_size:.1f} KB)")
print(f"   - Report: {report_file}")
print("="*70)

In [ ]:
# ====================================================================
# CELL 8: Display Sample Results
# ====================================================================

print("\n📋 Sample References (first 10):")
print("="*70)

for i, ref in enumerate(unique_refs[:10], 1):
    time = ref['timestamp_ms'] / 1000
    method = ref['detection_method']
    conf = ref['confidence']
    
    print(f"\n[{i}] {time:.1f}s | {method} | confidence: {conf:.2f}")
    print(f"    Type: {ref['reference_type']}")
    print(f"    Text: {ref['text'][:120]}...")
    
    if ref.get('context_before'):
        print(f"    Before: {ref['context_before'][:80]}...")

if len(unique_refs) > 10:
    print(f"\n... and {len(unique_refs) - 10} more references")

print("\n" + "="*70)

In [ ]:
# ====================================================================
# CELL 9: Statistics & Visualization
# ====================================================================

import matplotlib.pyplot as plt

if len(unique_refs) > 0:
    # Reference timeline
    timestamps = [r['timestamp_ms'] / 1000 for r in unique_refs]
    confidences = [r['confidence'] for r in unique_refs]
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))
    
    # Timeline scatter plot
    colors = ['blue' if r['detection_method'] == 'regex' else 'orange' for r in unique_refs]
    ax1.scatter(timestamps, confidences, c=colors, alpha=0.6, s=100)
    ax1.axhline(y=CONFIG['similarity_threshold'], color='red', linestyle='--', alpha=0.5, label='Threshold')
    ax1.set_xlabel('Time (seconds)', fontsize=12)
    ax1.set_ylabel('Confidence', fontsize=12)
    ax1.set_title('Visual References Timeline', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(['Threshold', 'Regex', 'Semantic'])
    
    # Type distribution
    types = [r['reference_type'] for r in unique_refs]
    type_counts = {t: types.count(t) for t in set(types)}
    ax2.bar(type_counts.keys(), type_counts.values(), color='steelblue')
    ax2.set_xlabel('Reference Type', fontsize=12)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Reference Type Distribution', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/phase2_analysis_{video_id}.png", dpi=150, bbox_inches='tight')
    plt.show()
    
    print("📊 Visualization saved!")
else:
    print("⚠️  No references found to visualize")

# Summary stats
print(f"\n📈 Statistics:")
print(f"   Average time between refs: {np.mean(np.diff(timestamps)) if len(timestamps) > 1 else 0:.1f}s")
print(f"   Reference density: {len(unique_refs) / (phase1_data['duration_ms']/1000) * 60:.1f} refs/minute")

print("\n✅ All done! Download files from /kaggle/working/output/")